<p style="float: left;"><a href="mixin-class-composition.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="nested-functions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Higher-order Functions

Higher order functions take other functions as parameters or return a function as
a result. This is possible because functions are first-class values in Scala.
The terminology can get a bit confusing at this point, and we use the phrase
"higher order function" for both methods and functions that take functions as parameters
or that return a function.

One of the most common examples is the higher-order
function `map` which is available for collections in Scala.

In [1]:
val salaries = Seq(20000, 70000, 40000)
val doubleSalary = (x: Int) => x * 2
val newSalaries = salaries.map(doubleSalary) // List(40000, 140000, 80000)

salaries: Seq[Int] = List(20000, 70000, 40000)
doubleSalary: Int => Int = ammonite.$sess.cmd0$Helper$$Lambda$2345/597765378@4a42c7d2
newSalaries: Seq[Int] = List(40000, 140000, 80000)

`doubleSalary` is a function which takes a single Int, `x`, and returns `x * 2`. In general, the tuple on the left of the arrow `=>` is a parameter list and the value of the expression on the right is what gets returned. On line 3, the function `doubleSalary` gets applied to each element in the
list of salaries.

To shrink the code, we could make the function anonymous and pass it directly as
an argument to map:

In [2]:
val salaries = Seq(20000, 70000, 40000)
val newSalaries = salaries.map(x => x * 2) // List(40000, 140000, 80000)

salaries: Seq[Int] = List(20000, 70000, 40000)
newSalaries: Seq[Int] = List(40000, 140000, 80000)

Notice how `x` is not declared as an Int in the above example. That's because the
compiler can infer the type based on the type of function map expects. An even more idiomatic way to write the same piece of code would be:

In [3]:
val salaries = Seq(20000, 70000, 40000)
val newSalaries = salaries.map(_ * 2)

salaries: Seq[Int] = List(20000, 70000, 40000)
newSalaries: Seq[Int] = List(40000, 140000, 80000)

Since the Scala compiler already knows the type of the parameters (a single Int),
 you just need to provide the right side of the function. The only
caveat is that you need to use `_` in place of a parameter name (it was `x` in
the previous example).

## Coercing methods into functions
It is also possible to pass methods as arguments to higher-order functions because
the Scala compiler will coerce the method into a function.

In [4]:
case class WeeklyWeatherForecast(temperatures: Seq[Double]) {

  private def convertCtoF(temp: Double) = temp * 1.8 + 32

  def forecastInFahrenheit: Seq[Double] = temperatures.map(convertCtoF) // <-- passing the method convertCtoF
}

defined class WeeklyWeatherForecast

Here the method `convertCtoF` is passed to `forecastInFahrenheit`. This is possible because the compiler coerces `convertCtoF` to the function `x => convertCtoF(x)` (note: `x` will
  be a generated name which is guaranteed to be unique within its scope).

## Functions that accept functions
One reason to use higher-order functions is to reduce redundant code. Let's say you wanted some methods that could raise someone's salaries by various factors. Without creating a higher-order function,
it might look something like this:

In [5]:
object SalaryRaiser {

  def smallPromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * 1.1)

  def greatPromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * math.log(salary))

  def hugePromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * salary)
}

defined object SalaryRaiser

Notice how each of the three methods vary only by the multiplication factor. To simplify,
you can extract the repeated code into a higher-order function like so:

In [6]:
object SalaryRaiser {

  private def promotion(salaries: List[Double], promotionFunction: Double => Double): List[Double] =
    salaries.map(promotionFunction)

  def smallPromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * 1.1)

  def bigPromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * math.log(salary))

  def hugePromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * salary)
}

defined object SalaryRaiser

The new method, `promotion`, takes the salaries plus a function of type `Double => Double`
(i.e. a function that takes a Double and returns a Double) and returns the product.

## Functions that return functions

There are certain cases where you want to generate a function. Here's an example
of a method that returns a function.

In [7]:
def urlBuilder(ssl: Boolean, domainName: String): (String, String) => String = {
  val schema = if (ssl) "https://" else "http://"
  (endpoint: String, query: String) => s"$schema$domainName/$endpoint?$query"
}

val domainName = "www.example.com"
def getURL = urlBuilder(ssl=true, domainName)
val endpoint = "users"
val query = "id=1"
val url = getURL(endpoint, query) // "https://www.example.com/users?id=1": String

defined function urlBuilder
domainName: String = "www.example.com"
defined function getURL
endpoint: String = "users"
query: String = "id=1"
url: String = "https://www.example.com/users?id=1"

Notice the return type of urlBuilder `(String, String) => String`. This means that
the returned anonymous function takes two Strings and returns a String. In this case,
the returned anonymous function is `(endpoint: String, query: String) => s"https://www.example.com/$endpoint?$query"`.
<p style="float: left;"><a href="mixin-class-composition.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="nested-functions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>